In [1]:
"""
run anaconda prompt 
"""

import os

home_path = os.path.expanduser('~')
cur_path = os.getcwd()
conda_path = home_path + "\\anaconda3"
conda_script_path = home_path + "\\anaconda3\\Scripts\\activate.bat"
exc = ' '.join(['start', '%windir%\System32\cmd.exe "/K"', conda_script_path, conda_path])
!$exc

"""
run tensorboard server

conda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs
http://localhost:6006/
"""

'\nrun tensorboard server\n\nconda activate py38-pytorch-gpu && tensorboard --port=6006 --logdir=runs\nhttp://localhost:6006/\n'

In [1]:
from manifest import Manifest
from logger import Logger
from remover import Remover

# 자동완성용 import
from env import Env
from runner import RunnerParams
from trainer import Trainer
from player import Player

from algorithms.dqn import DQN, DQNParams
from algorithms.dqn_runner import DQNRunner
from algorithms.ddqn import DDQN, DDQNParams
from algorithms.ddqn_runner import DDQNRunner
from algorithms.reinforce import Reinforce, ReinforceParams
from algorithms.reinforce_runner import ReinforceRunner
from algorithms.actorcritic import ActorCritic, ActorCriticParams
from algorithms.actorcritic_runner import ActorCriticRunner


In [9]:
import collections

def train():
    
    all_cases = Trainer().allcases
    cartpole_cases = filter(lambda x:x[0] == Env.CARTPOLE, all_cases)
    lunar_cases = filter(lambda x:x[0] == Env.LUNARLANDER, all_cases)

    check_interval_arr = [5]
    tr = Trainer(check_interval_arr)
    for env, algo in lunar_cases:
        tr.add_case(env, algo)
    tr.run()
    
    check_interval_arr = [10]
    tr = Trainer(check_interval_arr)
    for env, algo in cartpole_cases:
        tr.add_case(env, algo)
    tr.run()
    

def replay():
    import os
    from collections import defaultdict

    PATH = os.path.join(os.getcwd(), 'weights')
    filenames = ['.'.join(tokens[:-1]) 
                        for name in os.listdir(PATH) 
                        if (tokens := name.split('.')) and len(tokens) > 2]
    envs, algos = [], []
    for name in filenames:
        tokens = name.split('_')
        envs += [tokens[0]]
        algos += [tokens[1]]
    
    count = collections.defaultdict(int)

    for env, algo in zip(envs, algos):
        count[env] += 1
        count[algo] += 1
        

    print(*count.items(), sep='\n')

    Player('weights', filenames).run(debug=True)

replay()

# Remover().remove_dirs(['runs', 'weights', 'videos'])
# Remover().remove_dirs(['logs'])

# for i in range(10):
#     train()

# print_interval=10, max_epi=100 > 4분 35초
# print_interval=0, max_epi=100 > 4분 25초
# save_check_log=True, print_interval=0, max_epi=100 > 4분 37초

# for _ in range(10):
#     replay()

('ActorCritic', 18)
('CartPole-v1', 38)
('LunarLander-v2', 40)
('DDQN', 20)
('DQN', 20)
('Reinforce', 20)


In [ ]:
def tuning_reinforce():
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, Reinforce)]:
        start = 5
        k = 5
        for i in range(21):
            hparam = tr.default_hyperparam(env, algo)
            hparam.learning_rate = (start + k*i)/10000
            tr.add_case(env, algo, hparam)

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_actorcritic():
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, ActorCritic)]:
        start = 1
        k = 5
        for i in range(21):
            hparam = tr.default_hyperparam(env, algo)
            hparam.learning_rate = (start + k*i)/10000
            tr.add_case(env, algo, hparam)

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_dqn_node_cartpole():
    """
    8
    """
    tr = Trainer([20])
    for env, algo in [(Env.CARTPOLE, DQN)]:
        n_node = 2
        for i in range(8):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_dqn_node_lunarlander():
    """
    64~256
    """
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, DQN)]:
        n_node = 2
        for i in range(8):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_ddqn_node_cartpole():
    """
    8~16
    """
    tr = Trainer([20])
    for env, algo in [(Env.CARTPOLE, DDQN)]:
        n_node = 64
        for i in range(2):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

def tuning_ddqn_node_lunarlander():
    """
    32, 128, 256
    """
    tr = Trainer([10])
    for env, algo in [(Env.LUNARLANDER, DDQN)]:
        n_node = 32
        for i in range(4):
            hparam = tr.default_hyperparam(env, algo)
            hparam.n_node = n_node
            tr.add_case(env, algo, hparam)
            n_node *= 2

    runner_param_dic = {'save_net':True, 'max_episode':10000, 'print_interval':0, 'video_record_interval':0}
    tr.run(runner_param_dic)
    print('전체 테스트 종료')

# Remover().remove_dirs(['runs', 'weights', 'videos'])
# tuning_dqn_node_cartpole()
# tuning_dqn_node_lunarlander()
tuning_ddqn_node_lunarlander()